In [2]:
import pandas as pd
import numpy as np



In [3]:
df = pd.read_excel('teste intervalos.xlsx')
df.keys()

Index(['id', 'intervalo'], dtype='object')

In [4]:
def get_nanos_intervals(dataframe:pd.DataFrame):
    raw_intervals = dataframe['intervalo']
    prepared_intervals = [interval.replace(' ','').split('-') for interval in  raw_intervals]
    prepared_intervals = [[int(e[0]), int(e[1])] for e in prepared_intervals]

    change_moments = []
    for interval in prepared_intervals:
        change_moments.extend(interval)
    change_moments = list(set(change_moments))
    sorted_change_moments = sorted(list(map(lambda x : int(x),change_moments)))

    intervals = []
    for i ,moment in enumerate(sorted_change_moments):
        intervals.append((sorted_change_moments[-i-1],sorted_change_moments[-i]))
    intervals.reverse()
    return intervals

def is_valid_interval(interval,nano):
    [i1,i2] = interval
    [n1,n2] = nano
    i1 = (i1 - 24) if i1>i2 else i1
    c1 = (n1>=i1 and n2 <=i2)

    return c1


def discovery_nanos_from_intervals(dataframe:pd.DataFrame):
    raw_intervals = dataframe['intervalo']
    prepared_intervals = [interval.replace(' ','').split('-') for interval in  raw_intervals]
    prepared_intervals = [[int(e[0]), int(e[1])] for e in prepared_intervals]
    nanos = get_nanos_intervals(dataframe)

    nanos_dict = {}
    for i, interval in enumerate(prepared_intervals):
        nanos_dict[raw_intervals[i]] = [nano for nano in nanos if is_valid_interval(interval,nano)]

    return nanos_dict

discovery_nanos_from_intervals(df)

{'1-23': [(1, 2),
  (2, 3),
  (3, 4),
  (4, 6),
  (6, 7),
  (7, 12),
  (12, 22),
  (22, 23),
  (23, 1)],
 '1-4': [(1, 2), (2, 3), (3, 4), (23, 1)],
 '3-6': [(3, 4), (4, 6), (23, 1)],
 '7-12': [(7, 12), (23, 1)],
 '12-23': [(12, 22), (22, 23), (23, 1)],
 '4-23': [(4, 6), (6, 7), (7, 12), (12, 22), (22, 23), (23, 1)],
 '1-2': [(1, 2), (23, 1)],
 '4-6': [(4, 6), (23, 1)],
 '4-22': [(4, 6), (6, 7), (7, 12), (12, 22), (23, 1)],
 '23-4': [(1, 2), (2, 3), (3, 4), (23, 1)]}

In [10]:
new_df_model = {
    'id' : []
    ,'intervalo' : []
}

nanos_dict = discovery_nanos_from_intervals(df)

new_df = pd.DataFrame(new_df_model)
for id, intervalo in zip(df['id'], df['intervalo']):
    for nano in nanos_dict[intervalo]:
        new_row = [id,f'{nano[0]}-{nano[1]}']
        new_df.loc[len(new_df)] = new_row

In [11]:
new_df

,id,intervalo
0,a,1-2
1,a,2-3
2,a,3-4
3,a,4-6
4,a,6-7
5,a,7-12
6,a,12-22
7,a,22-23
8,a,23-1
9,b,1-2
